In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import create_map,concat_ws,size,array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [ ]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = input("spark.executor.instances:") or "64"
Num_core_per_executor = input("spark.executor.cores:") or "5"
Memory_executor = input("spark.executor.memory? Please end with 'g':") or "64g"
MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':") or "8g"

Num_core_per_driver = input("spark.driver.cores:") or "5"
Memory_driver = input("spark.driver.memory? Please end with 'g':") or "64g"
MemoryOverhead_driver = input("spark.driver.memoryOverhead? Please end with 'g':") or "8g"

Num_shuffle_partitions = input("spark.sql.shuffle.partitions:") or "400"

In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "Tetra_Spark_A1_BasedOn_D3_compts_pureSpark_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '128') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/path/to/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.config("spark.blacklist.enabled", "false") \
.getOrCreate()

spark_app_name: Tetra_Spark_A1_BasedOn_D3_compts_pureSpark_brain_11252025_1413


In [4]:
import graphframes
from graphframes import GraphFrame
from graphframes import *
from graphframes.lib import Pregel

### read input files

In [5]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

root
 |-- tetra_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



### globally obtain VT relation

In [6]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [7]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### globally obtain VF relation

In [8]:
# function to get VF from DF_T
def get_VF_init(df_tetra_order):
    df_VF_1 = df_tetra_order.withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VF_2 = df_tetra_order.withColumn("f1", sort_array(F.array("r2", "r1", "r3"), False)).withColumn("f2", sort_array(F.array("r2", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r2", "r3", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VF_3 = df_tetra_order.withColumn("f1", sort_array(F.array("r3", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r3", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r3", "r2", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VF_4 = df_tetra_order.withColumn("f1", sort_array(F.array("r4", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r4", "r1", "r3"), False)).withColumn("f3", sort_array(F.array("r4", "r2", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VF_union12 = df_VF_1.union(df_VF_2)
    df_VF_union123 = df_VF_union12.union(df_VF_3)
    df_VF_union1234 = df_VF_union123.union(df_VF_4)
    
    df_VF_init = df_VF_union1234.groupBy('r1').agg(collect_set('f1').alias('multi_f1'), collect_set('f2').alias('multi_f2'), collect_set('f3').alias('multi_f3'))
    return df_VF_init

df_VF_init = get_VF_init(df_tetra_order)

# df_VE_init.cache()

df_VF_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_f1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [9]:
# obtain VF relation
def get_VF(multi_f1, multi_f2, multi_f3):
# get_VF is used to obtain a complete VF relation from the partial VF relations
# multi_f1: partial VF relation
# multi_f2: partial VF relation
# multi_f3: partial VF relation

    faces = set()
    for f in multi_f1:
        faces.add(tuple(f))
        
    for f in multi_f2:
        faces.add(tuple(f))
        
    for f in multi_f3:
        faces.add(tuple(f))
    
    faces_list = sorted(faces) # save more time when using list(faces)
    
    return faces_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VF_udf = udf(get_VF, ArrayType(ArrayType(IntegerType())))

df_VF = df_VF_init.withColumn("VF", get_VF_udf(df_VF_init.multi_f1, df_VF_init.multi_f2, df_VF_init.multi_f3)).drop('multi_f1', 'multi_f2', 'multi_f3')
df_VF = df_VF.withColumnRenamed('r1', 'Ver')

df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### globally obtain VE relation

In [10]:
# function to get VE from DF_T
def get_VE_init(df_tetra_order):
    df_VE_1 = df_tetra_order.withColumn("e1", sort_array(F.array("r1", "r2"), False)).withColumn("e2", sort_array(F.array("r1", "r3"), False)).withColumn("e3", sort_array(F.array("r1", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VE_2 = df_tetra_order.withColumn("e1", sort_array(F.array("r2", "r1"), False)).withColumn("e2", sort_array(F.array("r2", "r3"), False)).withColumn("e3", sort_array(F.array("r2", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VE_3 = df_tetra_order.withColumn("e1", sort_array(F.array("r3", "r1"), False)).withColumn("e2", sort_array(F.array("r3", "r2"), False)).withColumn("e3", sort_array(F.array("r3", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VE_4 = df_tetra_order.withColumn("e1", sort_array(F.array("r4", "r1"), False)).withColumn("e2", sort_array(F.array("r4", "r2"), False)).withColumn("e3", sort_array(F.array("r4", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VE_union1234 = df_VE_1.union(df_VE_2).union(df_VE_3).union(df_VE_4)
    
    df_VE_init = df_VE_union1234.groupBy('r1').agg(collect_set('e1').alias('multi_e1'), collect_set('e2').alias('multi_e2'), collect_set('e3').alias('multi_e3'))
    return df_VE_init

df_VE_init = get_VE_init(df_tetra_order)

# df_VE_init.cache()

df_VE_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_e1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_e2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_e3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [11]:
# obtain VE relation
def get_VE(multi_e1, multi_e2, multi_e3):
# get_VE is used to obtain the partial VE relation
# multi_e1: partial VE relation
# multi_e2: partial VE relation
    edges = set()
    for e in multi_e1:
        edges.add(tuple(e))
        
    for e in multi_e2:
        edges.add(tuple(e))
        
    for e in multi_e3:
        edges.add(tuple(e))
    
    edges_list = sorted(edges) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return edges_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VE_udf = udf(get_VE, ArrayType(ArrayType(IntegerType())))

df_VE = df_VE_init.withColumn("VE", get_VE_udf(df_VE_init.multi_e1, df_VE_init.multi_e2, df_VE_init.multi_e3)).drop('multi_e1', 'multi_e2', 'multi_e3')
df_VE = df_VE.withColumnRenamed('r1', 'Ver')

df_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### globally obtain EF relation

In [12]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [13]:
# get edges from DF_T

df_TE = get_TE_from_T(df_tetra_order)

df_E1 = df_TE.select('e1')
df_E2 = df_TE.select('e2')
df_E3 = df_TE.select('e3')
df_E4 = df_TE.select('e4')
df_E5 = df_TE.select('e5')
df_E6 = df_TE.select('e6')

df_Edges = df_E1.union(df_E2).union(df_E3).union(df_E4).union(df_E5).union(df_E6)
df_Edges = df_Edges.withColumnRenamed('e1', 'edge')
df_Edges = df_Edges.distinct()
df_Edges.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [14]:
df_EV = df_Edges.withColumn('r1', df_Edges.edge[0]).withColumn('r2', df_Edges.edge[1])
df_EV.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)



##### join EV and VF

In [15]:
df_EF_init_1 = df_EV.join(df_VF, df_EV.r1==df_VF.Ver).select("edge", col("VF").alias("VF_1"), "r2")
df_EF_init_2 = df_EF_init_1.join(df_VF, df_EF_init_1.r2==df_VF.Ver).select("edge", "VF_1", col("VF").alias("VF_2"))

In [16]:
def get_EF(edge, VF_1, VF_2):
    EF = set()
    edge_set = set(edge)
    for face in VF_1:
        face_set = set(face)
        if edge_set.issubset(face_set):
            ET.add(tuple(face))
            
    for face in VF_2:
        face_set = set(face)
        if edge_set.issubset(face_set):
            ET.add(tuple(face))
            
    EF_list = list(EF)
    
    return EF_list

get_EF_udf = udf(get_EF, ArrayType(ArrayType(IntegerType())))

In [17]:
df_EF = df_EF_init_2.withColumn("EF", get_EF_udf(df_EF_init_2.edge, df_EF_init_2.VF_1, df_EF_init_2.VF_2)).select(col("edge").alias("Edge"), "EF")
df_EF.printSchema()

root
 |-- Edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- EF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### get EF where edges incident to the same vertex

In [18]:
df_VE_expl = df_VE.select("Ver", explode("VE").alias("single_edge"))

df_EF_expected_init = df_VE_expl.join(df_EF, df_VE_expl.single_edge == df_EF.Edge).drop('single_edge')
df_EF_expected_init.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- Edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- EF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [19]:
def union_EF(Edge, EF):
    EF_ary = [[Edge], EF]
    
    return EF_ary

union_EF_udf = udf(union_EF, ArrayType(ArrayType(ArrayType(IntegerType()))))

In [20]:
df_EF_expected_udf = df_EF_expected_init.withColumn("EF_expected", union_EF_udf(df_EF_expected_init.Edge, df_EF_expected_init.EF))
df_EF_expected_udf = df_EF_expected_udf.select("Ver", "EF_expected")

df_EF_expected = df_EF_expected_udf.groupby("Ver").agg(collect_list('EF_expected').alias('EF_final'))
df_EF_expected.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- EF_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



#### globally obtain FT relation

#### first get FV

In [21]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [22]:
# get faces from DF_T

df_TF = get_TF_from_T(df_tetra_order)

df_F1 = df_TF.select('f1')
df_F2 = df_TF.select('f2')
df_F3 = df_TF.select('f3')
df_F4 = df_TF.select('f4')

df_Faces = df_F1.union(df_F2).union(df_F3).union(df_F4)
df_Faces = df_Faces.withColumnRenamed('f1', 'face')
df_Faces = df_Faces.distinct()
df_Faces.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [23]:
df_FV = df_Faces.withColumn('r1', df_Faces.face[0]).withColumn('r2', df_Faces.face[1]).withColumn('r3', df_Faces.face[2])
df_FV.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)



##### then get VT, and join df_FV and df_VT

In [24]:
df_FT_init_1 = df_FV.join(df_VT, df_FV.r1==df_VT.Ver).select("face", col("VT").alias("VT_1"), "r2", "r3")
df_FT_init_2 = df_FT_init_1.join(df_VT, df_FT_init_1.r2==df_VT.Ver).select("face", "VT_1", col("VT").alias("VT_2"), "r3")
df_FT_init_3 = df_FT_init_2.join(df_VT, df_FT_init_2.r3==df_VT.Ver).select("face", "VT_1", "VT_2", col("VT").alias("VT_3"))

In [25]:
def get_FT(face, VT_1, VT_2, VT_3):
    FT = set()
    face_set = set(face)
    for tetra in VT_1:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    for tetra in VT_2:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    for tetra in VT_3:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    FT_list = list(FT)
    
    return FT_list

get_FT_udf = udf(get_FT, ArrayType(ArrayType(IntegerType())))

In [26]:
df_FT = df_FT_init_3.withColumn("FT", get_FT_udf(df_FT_init_3.face, df_FT_init_3.VT_1, df_FT_init_3.VT_2, df_FT_init_3.VT_3)).select(col("face").alias("Face"), "FT")
df_FT.printSchema()

root
 |-- Face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- FT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### get FT where faces incident to the same vertex

In [27]:
df_VF_expl = df_VF.select("Ver", explode("VF").alias("single_face"))

df_FT_expected_init = df_VF_expl.join(df_FT, df_VF_expl.single_face == df_FT.Face).drop('single_face')
df_FT_expected_init.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- Face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- FT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [28]:
def union_FT(Face, FT):
    FT_ary = [[Face], FT]
    
    return FT_ary

union_FT_udf = udf(union_FT, ArrayType(ArrayType(ArrayType(IntegerType()))))

In [29]:
df_FT_expected_udf = df_FT_expected_init.withColumn("FT_expected", union_FT_udf(df_FT_expected_init.Face, df_FT_expected_init.FT))
df_FT_expected_udf = df_FT_expected_udf.select("Ver", "FT_expected")

df_FT_expected = df_FT_expected_udf.groupby("Ver").agg(collect_list('FT_expected').alias('FT_final'))
df_FT_expected.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- FT_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



### concatenate the relations in one DataFrame

In [30]:
df_VT_VF = df_VT.join(df_VF, df_VT.Ver == df_VF.Ver)
df_VT_VF = df_VT_VF.select(df_VT.Ver, "VT", "VF")

df_VT_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [31]:
df_VT_VF_VE = df_VT_VF.join(df_VE, df_VT_VF.Ver == df_VE.Ver)
df_VT_VF_VE = df_VT_VF_VE.select(df_VT_VF.Ver, "VT", "VF", "VE")

df_VT_VF_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### concatenate the EF and FT relations in one DataFrame

In [32]:
df_VT_VF_VE_EF = df_VT_VF_VE.join(df_EF_expected, df_VT_VF_VE.Ver == df_EF_expected.Ver)

df_VT_VF_VE_EF = df_VT_VF_VE_EF.select(df_VT_VF_VE.Ver, "VT", "VF", "VE", "EF_final")
df_VT_VF_VE_EF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- EF_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



In [33]:
df_VT_VF_VE_EF_FT = df_VT_VF_VE_EF.join(df_FT_expected, df_VT_VF_VE_EF.Ver == df_FT_expected.Ver)

df_Forman_rela = df_VT_VF_VE_EF_FT.select(df_VT_VF_VE_EF.Ver, "VT", "VF", "VE", "EF_final", "FT_final")
df_Forman_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- EF_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |-- FT_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



### compute the edges, faces, and tetrahedra within a lower star

In [34]:
def get_LS_EFT(Ver, VT, VF, VE):
    LS_VT = []
    LS_VF = []    
    LS_VE = []   
    for tetra in VT:
        if Ver >= max(tetra):
            LS_VT.append(tetra)
    for face in VF:
        if Ver >= max(face):
            LS_VF.append(face)
    for edge in VE:
        if Ver >= max(edge):
            LS_VE.append(edge)
            
    return LS_VT, LS_VF, LS_VE

get_LS_EFT_schema = StructType([
    StructField("LS_tetra", ArrayType(ArrayType(IntegerType())),True), 
    StructField('LS_face',ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_edge',ArrayType(ArrayType(IntegerType())),True)
])

get_LS_EFT_udf = udf(get_LS_EFT, get_LS_EFT_schema)

In [44]:
df_LS_EFT_init = df_Forman_rela.withColumn("LS_EFT", get_LS_EFT_udf(df_Forman_rela.Ver, df_Forman_rela.VT, df_Forman_rela.VF, df_Forman_rela.VE))

df_LS_EFT = df_LS_EFT_init.select('Ver', col("VT"), col('LS_EFT.LS_tetra').alias('LS_VT'), col('LS_EFT.LS_face').alias('LS_VF'), col('LS_EFT.LS_edge').alias('LS_VE'))
df_LS_EFT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### compute Forman gradient

In [45]:
from collections import deque
def get_Forman(Ver, LS_tetra, LS_face, LS_edge):
    d = 3
    k = 0
    ST = deque() # ST stores k-simplices belonging to the lower star, the initial value is the vertex Ver
    ST.append(Ver)
    # LT = LS_face # LT stores the top simplices belonging to the lower star, here are faces
    crit_cell = [] # crit_cell will store critical simplices
    Forman_VE = [] # store VE pairs
    Forman_EF = [] # store EF pairs
    Forman_FT = [] # store FT pairs
    
    if (len(LS_edge)+len(LS_face)+len(LS_tetra)) == 0: # ver is a local minimum
        crit_cell.append([Ver])
        return crit_cell, Forman_VE, Forman_EF, Forman_FT
    
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_EF_inner):
        # compute the number of unpaired edges in tri_inner
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[0])
                
        potential_paired_edge = [edge0, edge1]
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge0)
            
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge1)
            
        return num_unpaired_edge, potential_paired_edge
    
    def get_EF_pair(ST_inner, crit_cell_inner, Forman_VE_inner, Forman_EF_inner):
        # this function is to get the EF pairs, E derives from CR while F derives from ST
        for i in range(len(ST_inner)):
            face = ST_inner[i]
            num_unpaired_edge, potential_paired_edge = num_unpaired_edges(face, crit_cell_inner, Forman_VE_inner, Forman_EF_inner)
            if num_unpaired_edge == 1:
                return [potential_paired_edge[0], face] # return the EF pair
            
    def num_unpaired_faces(tetra_inner, crit_cell_inner, Forman_vec_pair_EF_inner, Forman_vec_pair_FT_inner):
        # compute the number of unpaired faces in tetra_inner
        face0 = [tetra_inner[0], tetra_inner[1], tetra_inner[2]]
        face1 = [tetra_inner[0], tetra_inner[1], tetra_inner[3]]
        face2 = [tetra_inner[0], tetra_inner[2], tetra_inner[3]]
        
        num_unpaired_face = 3
        paired_face = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 3: # crit_inner_temp is a critical face
                paired_face.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_FT_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[0])
                
        potential_paired_face = [face0, face1, face2]
        if face0 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face0)
            
        if face1 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face1)
            
        if face2 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face2)
            
        return num_unpaired_face, potential_paired_face       
            
    def get_FT_pair(ST_inner, crit_cell_inner, Forman_EF_inner, Forman_FT_inner):
        # this function is to get the FT pairs, F derives from CR while T derives from ST
        for i in range(len(ST_inner)):
            tetra = ST_inner[i]
            num_unpaired_face, potential_paired_face = num_unpaired_faces(tetra, crit_cell_inner, Forman_EF_inner, Forman_FT_inner)
            if num_unpaired_face == 1:
                return [potential_paired_face[0], tetra] # return the FT pair
                
        
    
    while k <= d: # d == 3 for a tetrahedral mesh
        CR = ST # CR stores candidate critical simplices
        k = k + 1        
        if k == 1: # edges in the lower star
            ST = deque()
            for e in LS_edge:
                ST.append(e) # assign LS_edge to ST
        elif k == 2: # faces in the lower star
            ST = deque()
            for f in LS_face:
                ST.append(f) # assign LS_face to ST
        elif k == 3: # tetrahedra in the lower star
            ST = deque()
            for t in LS_tetra:
                ST.append(t) # assign LS_tetra to ST
        
        while len(CR) > 0:
            if k == 1: # get VE pairs
                pair_VE_V, pair_VE_E = CR.popleft(), ST.popleft()
                Forman_VE.append([pair_VE_V, pair_VE_E])
            elif k == 2: # get EF pairs
                pair_EF = get_EF_pair(ST, crit_cell, Forman_VE, Forman_EF)
                if pair_EF: # the EF pair exists
                    pair_EF_E = pair_EF[0]
                    pair_EF_F = pair_EF[1]
                    Forman_EF.append([pair_EF_E, pair_EF_F])
                    # remove the paired edges and faces
                    CR.remove(pair_EF_E)
                    ST.remove(pair_EF_F)
                else:
                    crit_cell.append(CR.popleft())
            elif k == 3: # get FT pairs
                pair_FT = get_FT_pair(ST, crit_cell, Forman_EF, Forman_FT)
                if pair_FT: # the FT pair exists
                    pair_FT_F = pair_FT[0]
                    pair_FT_T = pair_FT[1]
                    Forman_FT.append([pair_FT_F, pair_FT_T])
                    # remove the paired faces and tetra
                    CR.remove(pair_FT_F)
                    ST.remove(pair_FT_T)
                else:
                    crit_cell.append(CR.popleft())
                    
            elif k == 4:
                crit_cell.append(CR.popleft())
                
    return crit_cell, Forman_VE, Forman_EF, Forman_FT
                
# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for EF pairs        
add_multi_EF_schema = StructType([
    StructField("EF_pair_E", ArrayType(IntegerType()),True), 
    StructField('EF_pair_F',ArrayType(IntegerType()),True)
])

# StructType for FT pairs        
add_multi_FT_schema = StructType([
    StructField("FT_pair_F", ArrayType(IntegerType()),True), 
    StructField('FT_pair_T',ArrayType(IntegerType()),True)
])

# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("EF_pair", ArrayType(add_multi_EF_schema), True),
    StructField("FT_pair", ArrayType(add_multi_FT_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [46]:
df_Forman = df_LS_EFT.withColumn("LS_Forman", get_Forman_udf(df_LS_EFT.Ver, df_LS_EFT.LS_VT, df_LS_EFT.LS_VF, df_LS_EFT.LS_VE))
df_Forman = df_Forman.select(col("Ver"), col("VT"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- EF_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- EF_pair_F: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT_pair: array 

## Methods to build Graph FT

#### prepare nodes of Graph FT

In [47]:
df_tetra_node = df_tetra_order.select(col("r1"), col("r2"), col("r3"), col("r4"))
df_tetra_node = df_tetra_node.withColumn("tetra_origin", F.array("r1", "r2", "r3", "r4"))
df_tetra_node = df_tetra_node.withColumn("tetra", sort_array("tetra_origin", False)).drop("tetra_origin") # sort_array("tri", False), False means descending order
df_tetra_node = df_tetra_node.select(col("tetra"))
df_tetra_node.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [48]:
# add a special triangle [-1, -1, -1] as node -1; this is useful to deal with boundary elements
# e.g., for a boundary edge, it has only one coboundary triangle; to guarantee a consistent encoding, each edge has two coboundary triangels, the other coboundary triangle for a boundary edge is [-1,-1,-1]
# columns_special = ['tetra', 'V2_node']
value_special = [(-1, -1, -1, -1)]
df_special_V2_node = spark.createDataFrame(value_special, ArrayType(IntegerType()))
df_special_V2_node.printSchema()
# df_special_V2_node.show()

root
 |-- value: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [49]:
df_graph_V2_node = df_tetra_node.union(df_special_V2_node)
df_graph_V2_node = df_graph_V2_node.withColumn("tetra_str", concat_ws(",", col("tetra")))
df_graph_V2_node_final = df_graph_V2_node.select(col("tetra_str").alias("id"))
df_graph_V2_node_final.printSchema()
# df_graph_V2_node_final.show()

root
 |-- id: string (nullable = false)



In [50]:
# time cost to get df_graph_V2_node_final

t_df_graph_V2_node_final_0 = time.time()
num_df_graph_V2_node_final = df_graph_V2_node_final.count()

t_df_graph_V2_node_final_1 = time.time()
t_df_graph_V2_node_final = t_df_graph_V2_node_final_1 - t_df_graph_V2_node_final_0
print("time cost of df_graph_V2_node_final:", t_df_graph_V2_node_final)

time cost of df_graph_V2_node_final: 0.7373881340026855


#### prepare arcs of the Graph FT

In [51]:
df_Forman_FT_pair = df_Forman.select(col("Ver"), col("VT"), col("Forman.FT_pair").alias("FT_pair"))
df_Forman_FT_pair.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- FT_pair: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- FT_pair_F: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- FT_pair_T: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [52]:
# retrieve co-boundary tetrahedron (FT relation)
def get_co_tetra(VT, FT_pair):
    if len(FT_pair) > 0:
        co_tetra_pair = []
        num_FT_pair = len(FT_pair) # obtain the number of FT_pairs
        def Co_tetra(f,t,vt):
            # f is sorted
            for t_temp in vt:
                # e0 = sorted([vt[0], vt[1]], reverse=True) # sort the triangle in descending order
                f0 = [t_temp[0], t_temp[1], t_temp[2]] # f0, f1, and f2 are in descending order by default
                f1 = [t_temp[0], t_temp[1], t_temp[3]]
                f2 = [t_temp[0], t_temp[2], t_temp[3]]
                f3 = [t_temp[1], t_temp[2], t_temp[3]]
                t = [int(t[0]), int(t[1]), int(t[2]), int(t[3])]
                t_temp = [int(t_temp[0]), int(t_temp[1]), int(t_temp[2]), int(t_temp[3])]
                # if e==e0 or e==e1 or e==e2 and t != t_temp:
                if f==f0 or f==f1 or f==f2 or f==f3:
                    if t != t_temp:
                        return t_temp # t_temp is a coboundary triangle of t in vt
                
        for i in range(num_FT_pair):
            FT_pair_f = FT_pair[i]['FT_pair_F'] # the i-th FT_pair
            FT_pair_t = FT_pair[i]['FT_pair_T']
            FT_pair_co_t = Co_tetra(FT_pair_f, FT_pair_t, VT)
            if FT_pair_co_t is None: # len(FT_pair_co_t) is empty, boundary tetrahedron
                FT_pair_co_t = [-1, -1, -1, -1]
            co_tetra_pair.append([FT_pair_co_t, FT_pair_t])
            
        return co_tetra_pair
    
# the whole StructType
get_co_tetra_schema = StructType([
    StructField("FT_pair_co_t", ArrayType(IntegerType()), True),
    StructField("FT_pair_t", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tetra_udf = udf(get_co_tetra, ArrayType(get_co_tetra_schema))

df_Forman_V2_edge = df_Forman_FT_pair.withColumn("FT_pair_inTetra", get_co_tetra_udf(df_Forman_FT_pair.VT, df_Forman_FT_pair.FT_pair))
df_Forman_V2_edge = df_Forman_V2_edge.select(col("Ver"), col("FT_pair_inTetra"))
df_Forman_V2_edge.printSchema()
# df_Forman_V2_edge.show()

root
 |-- Ver: integer (nullable = true)
 |-- FT_pair_inTetra: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- FT_pair_co_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- FT_pair_t: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [53]:
# explode df_Forman_V2_edge
df_Forman_V2_edge_split = df_Forman_V2_edge.select(explode(df_Forman_V2_edge.FT_pair_inTetra).alias("FT_pair_src_dst"))

df_Forman_V2_edge_split = df_Forman_V2_edge_split.select(col("FT_pair_src_dst.FT_pair_co_t").alias("arc_dst"), col("FT_pair_src_dst.FT_pair_t").alias("arc_src")).na.drop()
df_Forman_V2_edge_split.printSchema()
# df_Forman_V2_edge_split.show()

root
 |-- arc_dst: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- arc_src: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [54]:
df_Forman_V2_edge_split = df_Forman_V2_edge_split.withColumn("arc_dst_str", concat_ws(",", col("arc_dst")))
df_Forman_V2_edge_split = df_Forman_V2_edge_split.withColumn("arc_src_str", concat_ws(",", col("arc_src")))
df_graph_V2_edge_final = df_Forman_V2_edge_split.select(col("arc_src_str").alias("src"), col("arc_dst_str").alias("dst"))

df_graph_V2_edge_final.printSchema()
# df_graph_V2_node_final.show()

root
 |-- src: string (nullable = false)
 |-- dst: string (nullable = false)



In [55]:
# time cost to get df_graph_V2_edge_final

t_df_graph_V2_edge_final_0 = time.time()
num_df_graph_V2_edge_final= df_graph_V2_edge_final.count()

t_df_graph_V2_edge_final_1 = time.time()
t_df_graph_V2_edge_final = t_df_graph_V2_edge_final_1 - t_df_graph_V2_edge_final_0
print("time cost of df_graph_V2_edge_final:", t_df_graph_V2_edge_final)

time cost of df_graph_V2_edge_final: 34.15576934814453


### construct Graph_FT

In [56]:
graph_FT = GraphFrame(df_graph_V2_node_final, df_graph_V2_edge_final)

print("type of graph_V2:", type(graph_FT))

type of graph_V2: <class 'graphframes.graphframe.GraphFrame'>


In [57]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

### Compute descending 3-manifolds (influence regions of maxima), which are the connected components of Graph_FT

In [58]:
t_con_0 = time.time()
# result_con consists of two columns, vertex id, component
result_con = graph_FT.connectedComponents()
result_con.printSchema()

t_con_1 = time.time()
t_con = t_con_1 - t_con_0
print("time cost of connected components:",t_con)

root
 |-- id: string (nullable = false)
 |-- component: long (nullable = true)

time cost of connected components: 124.49729084968567


#### Extract critical triangles and critical tetrahedra

In [8]:
file_df_Forman = 'Tetra_data' + '/' + tin_filename + '_Forman_VT' + '.parquet'

df_Forman = spark.read.parquet(file_df_Forman)
df_Forman.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- EF_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- EF_pair_F: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT_pair: array 

In [9]:
df_crit = df_Forman.select("Ver", "VT", col("Forman.Crit_cell").alias("Critical"))
print("number of partitions for df_crit:", df_crit.rdd.getNumPartitions())
df_crit.printSchema()
# df_crit.show()

number of partitions for df_crit: 312
root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [10]:
# split the Critical column to obtain the critical vertices, edges, and triangles
def get_crit_VEFT(crit_cell):
    if len(crit_cell) < 1: # crit_cell is empty, there are no critical simplices within this lower star
        return
    else:
        # initialize the critical vertices, critical edges, and critical triangles
        crit_ver = None
        crit_edge = []
        crit_tri = []
        crit_tetra = []
        for icrit_cell_temp in crit_cell:
            if len(icrit_cell_temp) == 1: # icrit_cell_temp stores a critical vertex
                crit_ver = icrit_cell_temp[0]
            if len(icrit_cell_temp) == 2: # icrit_cell_temp stores critical edges
                crit_edge.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 3: # icrit_cell_temp stores critical triangles
                crit_tri.append(icrit_cell_temp)
            if len(icrit_cell_temp) == 4: # icrit_cell_temp stores critical tetrahedra
                crit_tetra.append(icrit_cell_temp)
                
        if len(crit_edge) == 0:
            crit_edge = None
        if len(crit_tri) == 0:
            crit_tri = None
        if len(crit_tetra) == 0:
            crit_tetra = None
        return crit_ver, crit_edge, crit_tri, crit_tetra
        
# the whole StructType
get_crit_VEFT_schema = StructType([
    StructField("crit_ver", IntegerType(), True),
    StructField("crit_edge", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tri", ArrayType(ArrayType(IntegerType())), True),
    StructField("crit_tetra", ArrayType(ArrayType(IntegerType())), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_crit_VEFT_udf = udf(get_crit_VEFT, get_crit_VEFT_schema)

df_crit_VEFT = df_crit.withColumn("crit_cell", get_crit_VEFT_udf(df_crit.Critical))
print("number of partitions for df_crit_VEFT:", df_crit_VEFT.rdd.getNumPartitions())
df_crit_VEFT.printSchema()
# df_crit_VEFT.show()

number of partitions for df_crit_VEFT: 312
root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Critical: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- crit_cell: struct (nullable = true)
 |    |-- crit_ver: integer (nullable = true)
 |    |-- crit_edge: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tri: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- crit_tetra: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [11]:
# to get critical triangles
df_crit_F_init = df_crit_VEFT.select(col("Ver"), col("VT"), col("crit_cell.crit_tri").alias("Max_tri_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_F = df_crit_F_init.select(col("Ver"), col("VT"), explode("Max_tri_init").alias("Max_tri")).na.drop()
df_crit_F.printSchema()
print("number of partitions for df_crit_F:", df_crit_F.rdd.getNumPartitions())
# df_crit_F.show()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Max_tri: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_F: 312


In [12]:
# to get critical triangles
df_crit_T_init = df_crit_VEFT.select(col("Ver"), col("crit_cell.crit_tetra").alias("Max_tetra_init")).na.drop()

# there could be multiple critical triangles within one lower star
df_crit_T = df_crit_T_init.select(col("Ver"), explode("Max_tetra_init").alias("Max_tetra")).na.drop()
df_crit_T.printSchema()
print("number of partitions for df_crit_T:", df_crit_T.rdd.getNumPartitions())
# df_crit_T.show()

root
 |-- Ver: integer (nullable = true)
 |-- Max_tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)

number of partitions for df_crit_T: 312


In [13]:
# retrieve co-boundary tetrahedra (FT relation)
def get_co_tetra_of_tri(Max_tri, VT):
    if len(VT) > 0:
        f = Max_tri
        tetra_12 = []
        num_VT = len(VT) # obtain the number of FT_pairs
        for t_temp in VT:
            f0 = [t_temp[0], t_temp[1], t_temp[2]] # e0, e1, and e2 are in descending order by default
            f1 = [t_temp[0], t_temp[1], t_temp[3]]
            f2 = [t_temp[0], t_temp[2], t_temp[3]]
            f3 = [t_temp[1], t_temp[2], t_temp[3]]
            if f==f0 or f==f1 or f==f2 or f==f3:
                tetra_12.append(t_temp)
            # if len(tri_12) == 2: # Maybe this triangle is at the boundary, so it has only one corresponding triangle in VT
                # break
        '''
        if len(tetra_12) == 1:
            t_special = [-1,-1,-1,-1]
            tetra_12.append(t_special)
        '''
            
        return tetra_12
    
# the whole StructType
get_saddle_edge_tri_schema = StructType([
    StructField("Saddle_CoTetra_t1", ArrayType(IntegerType()), True),
    StructField("Saddle_CoTetra_t2", ArrayType(IntegerType()), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_co_tetra_of_tri_udf = udf(get_co_tetra_of_tri, ArrayType(ArrayType(IntegerType())))

df_crit_F_tetra = df_crit_F.withColumn("Saddle_CoTetra", get_co_tetra_of_tri_udf(df_crit_F.Max_tri, df_crit_F.VT))
df_crit_F_tetra = df_crit_F_tetra.select(col("Saddle_CoTetra"))
df_crit_F_tetra.printSchema()
# df_crit_E_tri.show()

root
 |-- Saddle_CoTetra: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [14]:
# obtain the co-boundary tetrahedra of 2-saddles
df_crit_F2tetra = df_crit_F_tetra.select(explode("Saddle_CoTetra").alias("Saddle_CoTetra_split"))
# some nodes may belong to multiple saddles, we need to remove them
df_crit_F2tetra = df_crit_F2tetra.distinct()
df_crit_F2tetra.printSchema()

root
 |-- Saddle_CoTetra_split: array (nullable = true)
 |    |-- element: integer (containsNull = true)



#### 1) left join result_con with 2-saddles tetra to identify the component that each saddles tetra belongs to

In [15]:
# identify the saddle tetrahedra that each component have, we also need to convert tetraheda data type from array of integers to string at first
df_crit_F2tetra = df_crit_F2tetra.withColumn("Saddle_CoTetra_split_str", concat_ws(",", col("Saddle_CoTetra_split"))).drop(col("Saddle_CoTetra_split"))

result_con_saddle_FT = result_con.join(df_crit_F2tetra,result_con.id==df_crit_F2tetra.Saddle_CoTetra_split_str, "inner")
result_con_saddle_FT = result_con_saddle_FT.withColumn("att", when(result_con_saddle_FT.Saddle_CoTetra_split_str.isNull() == False, result_con_saddle_FT.Saddle_CoTetra_split_str).otherwise(-1)).drop('Saddle_CoTetra_split_str')

print("number of partitions for result_con_saddle_FT:", result_con_saddle_FT.rdd.getNumPartitions())
result_con_saddle_FT.printSchema()

number of partitions for result_con_saddle_FT: 128
root
 |-- id: string (nullable = true)
 |-- component: long (nullable = true)
 |-- att: string (nullable = false)



In [16]:
# obtain all saddle tetrahedra for each connected component
unique_SdlTetra_per_con_FT = result_con_saddle_FT.groupBy('component').agg(collect_list('id').alias('multi_SdlTetra'))
unique_SdlTetra_per_con_FT.printSchema()

root
 |-- component: long (nullable = true)
 |-- multi_SdlTetra: array (nullable = true)
 |    |-- element: string (containsNull = true)



##### 2) left join df_graph_FT_arc with result_con to identify the connected component that each arc belongs to

In [17]:
# left join df_graph_V1_arc with result_con to identify which component that each arc belongs to
df_G_V2_arc_component = df_graph_FT_arc.join(result_con,df_graph_FT_arc.src==result_con.id, "left").drop('id')
df_G_V2_arc_component.printSchema()

root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- component: long (nullable = true)



##### 3) compute 1-manifolds through Lable Propagation Algorithm

In [18]:
Num_subgraphs = 15
t_find_minmax_0 = time.time()

# Find the minimum and maximum values of "component" in result_con
sub_graph_tmp_min = result_con.agg({"component": "min"}).collect()[0][0]
sub_graph_tmp_max = result_con.agg({"component": "max"}).collect()[0][0]

sub_graph_list = []
for i in range(Num_subgraphs+1):
    sub_graph_tmp = int(i * (sub_graph_tmp_max - sub_graph_tmp_min)/(Num_subgraphs) + sub_graph_tmp_min)
    sub_graph_list.append(sub_graph_tmp)    

t_find_minmax_1 = time.time()
t_find_minmax = t_find_minmax_1 - t_find_minmax_0
print("time cost to find sub_graph_list:", t_find_minmax)

time cost to find sub_graph_list: 15.856176853179932


In [19]:
# Create empty DataFrame with a schema
schema_special = StructType([
    StructField('id', LongType(),True),
    StructField("component", LongType(),True),
    StructField("att", LongType(),True),   
    StructField("label", LongType(),True)   
])

df_empty = spark.createDataFrame([], schema_special)

In [20]:
# set a checkpoint directory to improve performance
# Checkpointing regularly helps recover from failures, clean shuffle files, shorten the
# lineage of the computation graph, and reduce the complexity of plan optimization.

spark.sparkContext.setCheckpointDir('checkpoints')

In [21]:
# construct subgraphs and union the LPA results
t_LPA_total_0 = time.time()

sub_time = [] # to store time cost for each subgraph
for i in range(len(sub_graph_list) - 1):
    t_LPA_sub_0 = time.time()
    sub_graph_tmp_0 = sub_graph_list[i]
    sub_graph_tmp_1 = sub_graph_list[i+1]
    
    if i == len(sub_graph_list) - 1: # the last subgraphs should contain sub_graph_tmp_max
        sub_graph_node = result_con_saddle_FT.filter((result_con_saddle_FT.component >= sub_graph_tmp_0) & (result_con_saddle_FT.component <= sub_graph_tmp_1))
        sub_graph_arc = df_G_V2_arc_component.filter((df_G_V2_arc_component.component >= sub_graph_tmp_0) & (df_G_V2_arc_component.component <= sub_graph_tmp_1)).drop('id')
    else:
        sub_graph_node = result_con_saddle_FT.filter((result_con_saddle_FT.component >= sub_graph_tmp_0) & (result_con_saddle_FT.component < sub_graph_tmp_1))
        # sub_graph_node.printSchema() # 3 columns: id, component, att
        sub_graph_arc = df_G_V2_arc_component.filter((df_G_V2_arc_component.component >= sub_graph_tmp_0) & (df_G_V2_arc_component.component < sub_graph_tmp_1)).drop('id')
        # sub_graph_arc.printSchema() # 3 columns: src, dst, component
    
    sub_graph = GraphFrame(sub_graph_node, sub_graph_arc)
    
    sub_graph_LPA = sub_graph.pregel \
         .setMaxIter(10) \
         .withVertexColumn("label", col("att"), \
             Pregel.msg()) \
         .sendMsgToDst(when(Pregel.dst("label")>0, Pregel.dst("label")).otherwise(Pregel.src("label"))) \
         .aggMsgs(F.max(Pregel.msg())) \
         .run()
    t_LPA_sub_1 = time.time()
    df_empty = df_empty.union(sub_graph_LPA)
    print("Time cost for LPA to subgraph %ld: %f" % (sub_graph_tmp_0, t_LPA_sub_1 - t_LPA_sub_0))
    sub_time.append(t_LPA_sub_1 - t_LPA_sub_0)
    
t_LPA_total_1 = time.time()
print("*******************************************************")
print("Total time cost for LPA:", t_LPA_total_1 - t_LPA_total_0)

Time cost for LPA to subgraph 0: 997.309735
Time cost for LPA to subgraph 37795715307: 563.466075
Time cost for LPA to subgraph 75591430614: 564.135789
Time cost for LPA to subgraph 113387145921: 549.230922
Time cost for LPA to subgraph 151182861228: 545.990998
Time cost for LPA to subgraph 188978576536: 550.437645
Time cost for LPA to subgraph 226774291843: 548.651598
Time cost for LPA to subgraph 264570007150: 563.747358
Time cost for LPA to subgraph 302365722457: 561.152405
Time cost for LPA to subgraph 340161437764: 548.307344
Time cost for LPA to subgraph 377957153072: 554.905908
Time cost for LPA to subgraph 415752868379: 549.225694
Time cost for LPA to subgraph 453548583686: 544.421701
Time cost for LPA to subgraph 491344298993: 536.039540
Time cost for LPA to subgraph 529140014300: 521.237876
*******************************************************
Total time cost for LPA: 8698.437216997147
